# Demand Prediction Model
Train a Random Forest model to predict demand at each stop/time.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
data = pd.read_csv('../data/synthetic_demand.csv')
data.head()

In [ ]:
# Feature Engineering
X = data[["route_id", "stop_id", "hour", "weekday"]]
y = data["passenger_count"]

X = pd.get_dummies(X, columns=["route_id", "stop_id"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
model = RandomForestRegressor(n_estimators=200, max_depth=10)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f"MAE: {mae:.2f} | RMSE: {rmse:.2f}")

In [ ]:
joblib.dump(model, "../models/demand_model.pkl")

In [ ]:
import matplotlib.pyplot as plt

# Visualize feature importances for the Random Forest model
importances = model.feature_importances_
plt.figure(figsize=(8, 6))
plt.barh(X.columns, importances)
plt.xlabel("Importance")
plt.title("Feature Importance (Random Forest)")
plt.tight_layout()
plt.show()